In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "DEV")
dbutils.widgets.text("target_dataset", "VCM_DMT_PRD")
dbutils.widgets.text("catalog_name", "udp_wcm_dev")
dbutils.widgets.text("metadata_schema", "udp_wcm_metadata_dev")
dbutils.widgets.text("target_table", "a_billcnt_store")
dbutils.widgets.text(
    "dependency_table","VCM_DWH_PRD.F_TRANSACTION_LINE, VCM_STG_INF.SAP_VCDEXT01, VCM_DMT_PRD.A_FINANCE_BILL_SLL_DETAIL"
)

In [0]:
dbutils.widgets.text("field_calday", "CALDAY")
dbutils.widgets.text("field_id", "HASH_ID")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
environment = dbutils.widgets.get("environment")
target_dataset = dbutils.widgets.get("target_dataset")
target_table = dbutils.widgets.get("target_table")
metadata_schema = dbutils.widgets.get("metadata_schema")
dependency_table = dbutils.widgets.get("dependency_table")
dependency_table = [x.strip().upper() for x in dependency_table.split(",")]
dependency_table = "'" + "','".join(dependency_table) + "'"

print(f"environment: {environment}")
print(f"target_dataset: {target_dataset}")
print(f"target_table: {target_table}")
print(f"catalog_name: {catalog_name}")
print(f"metadata_schema: {metadata_schema}")
print(f"dependency_table: {dependency_table}")

In [0]:
field_calday = dbutils.widgets.get("field_calday")
field_id = dbutils.widgets.get("field_id")

print(f"field_calday: {field_calday}")
print(f"field_id: {field_id}")

In [0]:
%run "../common/common_etl_load"

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_store(
calday DATE,
store_id STRING,
store_name STRING,
billcnt_total BIGINT,
billcnt_excl_sll BIGINT,
billcnt_promo BIGINT,
billcnt_promo_5mch BIGINT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
if CALDAY_IN_STR != '':
    spark.sql(f"""
    create or replace TEMP VIEW temp_a_billcnt_store as
    WITH 
    transaction_sll AS (
        SELECT
            DISTINCT
            calday,
            plant store_id,
            rpa_tnr transaction_no
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vcdext01
        WHERE _bic_zc_fgroup = 'OGID'
            AND _bic_zc_fname = 'B2B'
            AND _bic_zc_fvalue LIKE 'B2B%'
    ),
    store_txn AS (
        SELECT
            a.calday,
            a.store_id,
            a.transaction_type,
            COUNT(DISTINCT transaction_id) m_txn_cnt,
            COUNT(DISTINCT IF(b.calday IS NULL AND c1.calday IS NULL, transaction_id, NULL)) m_txn_cnt_excl_sll,
            COUNT(DISTINCT IF(b.calday IS NULL AND c1.calday IS NULL AND product.mch2_id IN ('101', '102', '201', '202', '203'), transaction_id, NULL))m_txn_cnt_excl_sll_5mch,
            COUNT(DISTINCT CASE WHEN promotion_sale > 0 THEN transaction_id END) m_txn_cnt_promo,
            COUNT(DISTINCT CASE WHEN promotion_sale > 0
                                    AND product.mch2_id IN ('101', '102', '201', '202', '203')
                                    THEN transaction_id END) m_txn_cnt_promo_5mch
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_transaction_line a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product product ON a.product_id = product.product_id
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dmt.a_finance_bill_sll_detail b ON a.calday = b.calday AND a.store_id = b.store_id AND a.product_id = b.product_id AND LTRIM('0', a.transaction_no) = LTRIM('0', b.transaction_no)
        LEFT JOIN transaction_sll c1 ON a.calday = c1.calday AND a.store_id = c1.store_id AND LTRIM('0', a.transaction_no) = LTRIM('0', c1.transaction_no)
        WHERE a.CALDAY IN ({CALDAY_IN_STR})
        GROUP BY 1,2,3
    ),
    bill_cnt_by_str AS (
        SELECT
            calday,
            a.store_id,
            c.store_name,
            SUM(b.bill_count * a.m_txn_cnt) billcnt_total,
            SUM(b.bill_count * a.m_txn_cnt_excl_sll) billcnt_excl_sll,
            SUM(b.bill_count * a.m_txn_cnt_excl_sll_5mch) billcnt_excl_sll_5mch,
            SUM(b.bill_count * a.m_txn_cnt_promo) billcnt_promo,
            SUM(b.bill_count * a.m_txn_cnt_promo_5mch) billcnt_promo_5mch
        FROM 
            store_txn a
            INNER JOIN {catalog_name}.udp_wcm_silver_dim.transaction_type b ON a.transaction_type = b.transaction_type
            INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store c ON a.store_id = c.store_id
        GROUP BY 1,2,3
    )
    SELECT * FROM bill_cnt_by_str
    """)
else:
    print(f"No valid calday's found In common-etl to create the view: temp_a_billcnt_store")

In [0]:
# Get the calday list string from temp table
calday_query = f"""
SELECT
    CONCAT("'", CONCAT_WS("','", COLLECT_SET(CAST(TO_DATE(calday, 'yyyy-MM-dd') AS STRING))), "'") AS calday_in_str
FROM temp_a_billcnt_store
"""

calday_df = spark.sql(calday_query)
calday_in_str = calday_df.collect()[0][0]

# Only run DELETE if the string is non-empty
if calday_in_str.strip("'"):  # checks if there's any date inside the quotes
    spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_store
    WHERE calday IN ({calday_in_str})
    """)
else:
    print("No valid CALDAYs found to delete.")

In [0]:
if CALDAY_IN_STR != "":
    # Insert rows from temp table into main table
    INSERT_ROW = ", ".join(
        spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_store").columns
    )
    spark.sql(
        f"""
        INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_store ({INSERT_ROW})
        SELECT {INSERT_ROW} FROM temp_{target_table}
    """
    )
else:
    print(
        f"temp_{target_table} could't be created as no valid calday's found In common-etl so {catalog_name}.udp_wcm_gold_vcm_dmt.a_billcnt_store can't be inserted using calday's from temp_{target_table}"
    )

In [0]:
if CALDAY_IN_STR == '':
    dbutils.notebook.exit("No valid calday's found In common-etl to insert data into Table: ETL_LOG_DEPENDENCIES and ETL_DELTA_TABLE and ")

In [0]:
%run "../common/common_etl_update"

In [0]:
spark.sql(f"""DROP VIEW IF EXISTS temp_{target_table}""")